# Precomputing values for use in fits of Stan models

Because of the way Stan works, it is necessary to compute some values in advance which can then be passed into the fit an interpolated over. The precomputed values will be different for different sets of source distances, and therefore different catalogues. 

Here we show how to compute the values for the SBG catalogue, but it is exactly the same for all cases, just changing the input label.

For ease, all the precomputed table files used are provided for use in this repository.

In [4]:
import os
import h5py
import numpy as np

from fancy import Data, Model, Analysis

detector_type = "auger2014"

In [5]:
# change detector property imports based on detector type string
if detector_type == "TA2015":
    from fancy.detector.TA2015 import detector_properties, Eth
elif detector_type == "auger2014":
    from fancy.detector.auger2014 import detector_properties, Eth
elif detector_type == "auger2010":
    from fancy.detector.auger2010 import detector_properties, Eth
else:
    raise Exception("Undefined detector type!")

In [6]:
# Define file containing catalogue information
source_file = '../data/sourcedata.h5'

# Path to Stan files
stan_path = '../stan/'

# make output directory if it doesnt exist
if not os.path.isdir("output"):
    os.mkdir("output")

# source_type = "2FHL_250Mpc"
# source_type = "swift_BAT_213"
source_types = ["SBG_23", "2FHL_250Mpc", "swift_BAT_213"]

for source_type in source_types:
    print("Current Source: {0}".format(source_type))

    # File in which to store precomputation
    # create new files with TA label
    # table_file = 'output/precomputation_storage.h5'
    # table_file = 'output/precomputation_storage_TA.h5'
    table_file = "tables_{0}_{1}.h5".format(source_type, detector_type)

    data = Data()
    data.add_source(source_file, source_type) 
    # data.source.select_sources([0, 1]) # just select 2 sources to speed up demo
    data.add_detector(detector_properties)  # KW: add detector information

    model_name = 'joint_model.stan'
    model = Model(model_filename = model_name, include_paths = stan_path)
    model.input(Eth = Eth) # EeV
    # model.input(Eth = 57) # EeV

    # precomp_output = 'output/testing_precomputation.h5'
    # precomp_output = 'output/testing_precomputation_TA.h5'
    # summary = b'Demonstration of precomputation.' 
    summary = b'Precomputation for source catalogues'
    # summary = b'Precomputation for 2FHL catalogue with TA observatory'
    # summary = b'Precomputation for swift_bat catalogue with TA observatory'
    analysis = Analysis(data, model, analysis_type = 'joint', 
                        filename = table_file, summary = summary)

    print("Evaluating Exposure Integral...")
    analysis.build_tables(fit_only = True)
    analysis.tables.save(table_file)


    print("Evaluating Arrival Energies...")
    analysis.build_energy_table(table_file = table_file)

Current Source: SBG_23


Precomputing exposure integral: 100%|██████████| 23/23 [01:40<00:00,  4.39s/it]


Precomputing exposure integral: 100%|██████████| 23/23 [00:51<00:00,  2.23s/it]


Current Source: 2FHL_250Mpc


Precomputing exposure integral: 100%|██████████| 17/17 [01:34<00:00,  5.57s/it]


Precomputing exposure integral: 100%|██████████| 17/17 [00:57<00:00,  3.39s/it]


Current Source: swift_BAT_213


Precomputing exposure integral: 100%|██████████| 213/213 [12:43<00:00,  3.58s/it]


Precomputing exposure integral: 100%|██████████| 213/213 [07:02<00:00,  1.98s/it]
